# Questions
- I was forced to drop all the NaN values when running cell # 11 ( split the data to train and test)
- My decision tree actually gives the same result and takes longer. What am I missing?

# Using loan dataset to learn random forest
the data was taken from https://www.lendingclub.com/info/download-data.action  (year 2015)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import ensemble
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import timeit

In [2]:
loan_df = pd.DataFrame()
loan_df = pd.read_csv('LoanStats2015.csv')
loan_df.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,24700.0,24700.0,24700.0,36 months,11.99%,820.28,C,C1,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,35000.0,35000.0,35000.0,60 months,14.85%,829.90,C,C5,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,3600.0,3600.0,3600.0,36 months,13.99%,123.03,C,C4,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,10400.0,10400.0,10400.0,60 months,22.45%,289.91,F,F1,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,20000.0,20000.0,20000.0,60 months,10.78%,432.66,B,B4,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# dropping columns with 85% or more NaN values.
i = 0
for col in loan_df:
    if(loan_df[col].isnull().sum() >= 0.85 * len(loan_df)):
        loan_df.drop(columns=col, inplace=True)
        i += 1
print("%d total columns were removed" % i)
loan_df.head()

53 total columns were removed


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,24700.0,24700.0,24700.0,36 months,11.99%,820.28,C,C1,Engineer,10+ years,...,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,N,N
1,35000.0,35000.0,35000.0,60 months,14.85%,829.90,C,C5,Information Systems Officer,10+ years,...,100.0,0.0,0.0,0.0,381215.0,52226.0,62500.0,18000.0,N,N
2,3600.0,3600.0,3600.0,36 months,13.99%,123.03,C,C4,leadman,10+ years,...,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,N,N
3,10400.0,10400.0,10400.0,60 months,22.45%,289.91,F,F1,Contract Specialist,3 years,...,96.6,60.0,0.0,0.0,439570.0,95768.0,20300.0,88097.0,N,N
4,20000.0,20000.0,20000.0,60 months,10.78%,432.66,B,B4,truck driver,10+ years,...,100.0,50.0,0.0,0.0,218418.0,18696.0,6200.0,14877.0,N,N


In [4]:
# remove rows with all values = NaN
print("Before dropping rows", loan_df.shape)
loan_df.dropna(how='all', inplace=True)
print("After dropping rows", loan_df.shape)

Before dropping rows (421099, 91)
After dropping rows (421095, 91)


In [5]:
# check the columns type
loan_df.dtypes

loan_amnt                         float64
funded_amnt                       float64
funded_amnt_inv                   float64
term                               object
int_rate                           object
installment                       float64
grade                              object
sub_grade                          object
emp_title                          object
emp_length                         object
home_ownership                     object
annual_inc                        float64
verification_status                object
issue_d                            object
loan_status                        object
pymnt_plan                         object
purpose                            object
title                              object
zip_code                           object
addr_state                         object
dti                               float64
delinq_2yrs                       float64
earliest_cr_line                   object
inq_last_6mths                    

In [6]:
# before removing categorical columns with more than 30 unique values, check which one looks important
categorical = loan_df.select_dtypes(include=['object'])
for c in categorical:
    if(categorical[c].nunique() > 30):
        print("Column: ", c, "has {} unique values".format(categorical[c].nunique()))
        print(loan_df[c].head(), "\n")

Column:  int_rate has 111 unique values
0    11.99%
1    14.85%
2    13.99%
3    22.45%
4    10.78%
Name: int_rate, dtype: object 

Column:  sub_grade has 35 unique values
0    C1
1    C5
2    C4
3    F1
4    B4
Name: sub_grade, dtype: object 

Column:  emp_title has 120811 unique values
0                       Engineer
1    Information Systems Officer
2                        leadman
3            Contract Specialist
4                   truck driver
Name: emp_title, dtype: object 

Column:  zip_code has 914 unique values
0    577xx
1    076xx
2    190xx
3    174xx
4    605xx
Name: zip_code, dtype: object 

Column:  addr_state has 49 unique values
0    SD
1    NJ
2    PA
3    PA
4    IL
Name: addr_state, dtype: object 

Column:  earliest_cr_line has 668 unique values
0    Dec-99
1    Sep-08
2    Aug-03
3    Jun-98
4    Aug-00
Name: earliest_cr_line, dtype: object 

Column:  revol_util has 1211 unique values
0    19.20%
1    11.60%
2    29.70%
3    64.50%
4    56.20%
Name: revol_util, dt

In [7]:
# from above, it looks like 'int_rate' and 'revol_util' can be change to numeric. the rest will be removed.
loan_df['int_rate'] = pd.to_numeric(loan_df['int_rate'].str.strip('%'))
loan_df['revol_util'] = pd.to_numeric(loan_df['revol_util'].str.strip('%'))

# make 'term' also numeric
loan_df['term'] = pd.to_numeric(loan_df['term'].str.strip('months'))

In [8]:
# drop categorical columns with more than 30 unique values
categorical = loan_df.select_dtypes(include=['object'])
for col in categorical:
    if(loan_df[col].nunique() > 30):
        print("{} was dropped".format(col))
        loan_df.drop(columns=col, inplace=True)
        
loan_df.head()        

sub_grade was dropped
emp_title was dropped
zip_code was dropped
addr_state was dropped
earliest_cr_line was dropped
last_pymnt_d was dropped
last_credit_pull_d was dropped


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,24700.0,24700.0,24700.0,36,11.99,820.28,C,10+ years,MORTGAGE,65000.0,...,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,N,N
1,35000.0,35000.0,35000.0,60,14.85,829.90,C,10+ years,MORTGAGE,110000.0,...,100.0,0.0,0.0,0.0,381215.0,52226.0,62500.0,18000.0,N,N
2,3600.0,3600.0,3600.0,36,13.99,123.03,C,10+ years,MORTGAGE,55000.0,...,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,N,N
3,10400.0,10400.0,10400.0,60,22.45,289.91,F,3 years,MORTGAGE,104433.0,...,96.6,60.0,0.0,0.0,439570.0,95768.0,20300.0,88097.0,N,N
4,20000.0,20000.0,20000.0,60,10.78,432.66,B,10+ years,MORTGAGE,63000.0,...,100.0,50.0,0.0,0.0,218418.0,18696.0,6200.0,14877.0,N,N


In [9]:
# see the unique categorical values
categorical = loan_df.select_dtypes(include=['object'])
for c in categorical:
    print(c)
    print(categorical[c].unique(), "\n")

grade
['C' 'F' 'B' 'E' 'A' 'D' 'G'] 

emp_length
['10+ years' '3 years' '4 years' '1 year' '7 years' '6 years' '5 years'
 '8 years' '2 years' '9 years' '< 1 year' nan] 

home_ownership
['MORTGAGE' 'RENT' 'OWN' 'ANY'] 

verification_status
['Not Verified' 'Source Verified' 'Verified'] 

issue_d
['Dec-15' 'Nov-15' 'Oct-15' 'Sep-15' 'Aug-15' 'Jul-15' 'Jun-15' 'May-15'
 'Apr-15' 'Mar-15' 'Feb-15' 'Jan-15'] 

loan_status
['Fully Paid' 'Current' 'Charged Off' 'Late (31-120 days)'
 'Late (16-30 days)' 'In Grace Period' 'Default'] 

pymnt_plan
['n' 'y'] 

purpose
['small_business' 'debt_consolidation' 'major_purchase' 'home_improvement'
 'other' 'credit_card' 'house' 'vacation' 'car' 'medical' 'moving'
 'renewable_energy' 'wedding' 'educational'] 

title
['Business' 'Debt consolidation' 'Major purchase' nan
 'Credit card refinancing' 'Home improvement' 'Other' 'Home buying'
 'Vacation' 'Car financing' 'Medical expenses' 'Moving and relocation'
 'Green loan' 'odymeds' 'SAVE' 'Learning and train

In [10]:
# dropping al NaN values
loan_df = loan_df.dropna()

In [11]:
# separating to dependent and independent
X = loan_df.drop('loan_status', 1)
y = loan_df['loan_status']

# create dummy variables 
X = pd.get_dummies(X)
y = pd.get_dummies(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [12]:
# decision tree
decision_tree = tree.DecisionTreeClassifier(
    criterion='entropy',
#    max_features=1,
#    max_depth=4,
)

start = timeit.default_timer()

decision_tree.fit(X_train, y_train)
decision_tree.score(X_test, y_test) 

print(cross_val_score(decision_tree, X, y, cv=10))

stop = timeit.default_timer()
print('Time for decision tree: ', stop - start) 

[0.97014925 0.97803163 0.97539543 0.98506151 0.98066784 0.98506151
 0.98769772 0.98154657 0.98506151 0.98594025]
Time for decision tree:  6.2719219999999964


In [13]:
rfc = ensemble.RandomForestClassifier()

start = timeit.default_timer()

rfc.fit(X_train, y_train)
rfc.score(X_test, y_test)

print(cross_val_score(rfc, X, y, cv=10))

stop = timeit.default_timer()
print('Time for random forest: ', stop - start)  

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[0.97541703 0.97539543 0.98242531 0.98242531 0.9797891  0.97891037
 0.98154657 0.97627417 0.9771529  0.98506151]
Time for random forest:  3.657275200000001
